In [24]:
import keras
from keras.datasets import boston_housing
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import numpy as np

In [21]:
batch_size = 32
epochs = 5

In [13]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [25]:
order = np.argsort(np.random.random(y_train.shape))
x_train = x_train[order]
y_train = y_train[order]

In [26]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std


In [38]:
inp = keras.Input(shape=(13,))
x = Dense(50,activation='relu')(inp)
x = Dropout(0.2)(x)
x = Dense(50,activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(1)(x)
model = keras.Model(inp, out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                700       
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 3,301
Trainable params: 3,301
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
k = 10
num_val_samples = len(x_train) // k
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = x_train[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [x_train[:i * num_val_samples],
                            x_train[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [y_train[:i * num_val_samples],
                            y_train[(i+1)*num_val_samples:]],
                            axis=0)

    model.fit(partial_train_data,
              partial_train_targets,
              epochs=epochs,
              batch_size=16,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

Processing fold # 0
40/40 [==============================] - 0s 10ms/step
Processing fold # 1
40/40 [==============================] - 0s 127us/step
Processing fold # 2
40/40 [==============================] - 0s 548us/step
Processing fold # 3
40/40 [==============================] - 0s 398us/step
Processing fold # 4
40/40 [==============================] - 0s 530us/step
Processing fold # 5
40/40 [==============================] - 0s 223us/step
Processing fold # 6
40/40 [==============================] - 0s 139us/step
Processing fold # 7
40/40 [==============================] - 0s 553us/step
Processing fold # 8
40/40 [==============================] - 0s 512us/step
Processing fold # 9
40/40 [==============================] - 0s 378us/step


In [49]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [2.2697131872177123, 1.416119933128357, 1.8150000810623168, 1.6945310354232788, 1.4242849349975586, 1.903044295310974, 1.6378009080886842, 1.9217531204223632, 2.3140096187591555, 2.1862066268920897]
mean all scores : 1.858246374130249
